<a href="https://colab.research.google.com/github/hy103/NLP/blob/main/Binary_Classification_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
print(torch.__version__)

2.0.1+cu118


In [18]:
# pytorch mlp for binary classification
from numpy import vstack
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCELoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

In [19]:
# Custom dataset class
class CSVDataset(Dataset):
  #load the datset
  def __init__(self, path):
    df = read_csv(path, header = None)
    self.X = df.values[:, :-1]
    self.y = df.values[:, -1]
    self.X = self.X.astype('float32')
    self.y = LabelEncoder().fit_transform(self.y)
    self.y = self.y.astype('float32')
    self.y = self.y.reshape((len(self.y), 1))


  def __len__(self):
    return len(self.X)

  def __get_item__(self, idx):
    return [self.X[idx], self.y[idx]]

  def get_splits(self, n_test=0.3):
    test_size = round(n_test* len(self.X))
    train_size = len(self.X) - test_size

    return random_split(self, [train_size, test_size])



In [20]:
# Module definition
# MLP clas
class MLP(Module):
  # define model elements
  def __init__(self, n_inputs):
    super(MLP, self).__init__()

    # input to first hidden layer
    self.hidden1 = Linear(n_inputs, 10)
    kaiming_uniform_(self.hidden1.weight, nonlinearity = 'relu')
    self.act1 = ReLU()

    # Second hidden layer
    self.hidden2 = Linear(10,8)
    kaiming_uniform_(self.hidden2.weight, nonlinearity = 'relu')
    self.act2 = ReLU()

    # Third hidden layer
    self.hidden3 = Linear(8,1)
    kaiming_uniform_(self.hidden3.weight, nonlinearity = 'relu')
    self.act3 = Sigmoid()


    def forward(self, x):
      #input to first hidden layer
      X = self.hidden1(X)
      X = self.act1(X)
      #input to first hidden layer
      X = self.hidden2(X)
      X = self.act2(X)
      #input to first hidden layer
      X = self.hidden3(X)
      X = self.act3(X)
      return X

In [41]:
# Prepare datset
def prepare_data(path):
  dataset = CSVDataset(path)  
  train, test = dataset.get_splits()

  train_dl = DataLoader(train, batch_size = 32, shuffle = True)
  test_dl = DataLoader(test, batch_size= 1024, shuffle = False)
  return train_dl, test_dl

# train the model
def train_model(train_dl, model):
    # define the optimization
    criterion = BCELoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # enumerate epochs
    for epoch in range(100):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()

def evaluate_model(test_dl, model):
  predictions, actuals = list(), list() 
  for i, (inputs, targets) in enumerate(test_dl):
    yhat = model(inputs)
    yhat = yhat.detach().numpy()
    actual = targets.numpy()
    actual = actual.reshape((len(actual), 1))
    yhat = yhat.round()
    predictions.append(yhat)
    actuals.append(actual)
  predictions, actuals = vstack(predictions), vstack(actuals)
  acc = accuracy_score(actuals, predictions)
  return acc



def predict(row, model):
  row = Tensor([row])
  yhat = model(row)
  yhat = yhat.detach().numpy()
  return yhat


In [42]:
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv'

train_dl, test_dl = prepare_data(path)
print(len(train_dl.dataset), len(test_dl.dataset))
# define the network
model = MLP(34)
train_model(train_dl, model)
acc = evaluate_model(test_dl, model)
print('Accuracy : %.3f' %acc)
# make a single prediction (expect class=1)
row = [1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300]
yhat = predict(row, model)
print('Predicted: %.3f (class=%d)' % (yhat, yhat.round()))

235 116
Accuracy : 0.897
Predicted: 0.992 (class=1)
